In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [0]:
#!mkdir -p drive
#!google-drive-ocamlfuse drive
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目
#os.chdir("ColabData")
os.chdir("drive/ColabData") 
!ls

 00.hdf5	  099pred17.png   099pred2.png	  10-0.0148.hdf5
 05-0.0147.hdf5   099pred18.png   099pred35.png   10-0.0260.hdf5
 05-0.0259.hdf5   099pred19.png   099pred36.png   15-0.0259.hdf5
 099pred10.png	  099pred1.png	  099pred3.png	 '30-0.0155 (1).hdf5'
 099pred11.png	  099pred20.png   099pred4.png	  data_generator.py
 099pred12.png	  099pred21.png   099pred5.png	  drive
 099pred13.png	  099pred22.png   099pred6.png	  LOSS.png
 099pred14.png	  099pred23.png   099pred7.png	  __pycache__
 099pred15.png	  099pred24.png   099pred8.png
 099pred16.png	  099pred25.png   099pred9.png


In [0]:
!ls


 00.hdf5	  099pred17.png   099pred2.png	  10-0.0148.hdf5
 05-0.0147.hdf5   099pred18.png   099pred35.png   10-0.0260.hdf5
 05-0.0259.hdf5   099pred19.png   099pred36.png   15-0.0259.hdf5
 099pred10.png	  099pred1.png	  099pred3.png	 '30-0.0155 (1).hdf5'
 099pred11.png	  099pred20.png   099pred4.png	  data_generator.py
 099pred12.png	  099pred21.png   099pred5.png	  drive
 099pred13.png	  099pred22.png   099pred6.png	  LOSS.png
 099pred14.png	  099pred23.png   099pred7.png	  __pycache__
 099pred15.png	  099pred24.png   099pred8.png
 099pred16.png	  099pred25.png   099pred9.png


In [0]:
#%%
import matplotlib.pyplot as plt
import numpy as np
import glob
import skimage.io as io
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import cv2
def adjust_image(img):   
    if(np.max(img) > 1):
        img = img / 255.0
    return img

def adjust_mask(prediction,threshold):
    
    prediction[prediction>=threshold]=1
    prediction[prediction<threshold]=0
    prediction=np.uint8(prediction)

    return prediction

def slice_data(image,row,col,size_image):
    '''slice whole images into (row*col)*256*256 sub images '''
    image_array=[]
    for i in range(row):
        for j in range(col):
            image_array.append(image[i*size_image:(i+1)*size_image,j*size_image:(j+1)*size_image])
    image_array=np.array(image_array)
   
    return image_array


def read_images_slice(path,size_image,whole_tissue):
    all_images_array=[]
    all_images_path=glob.glob(path)
    num_images=len(all_images_path)
    if whole_tissue:
        for i in range(num_images):
           
            im=adjust_image(io.imread(path[:-1]+'ori('+str(i+26)+').png',as_gray=True))
            #print(path[:-1]+'ori('+str(i+1)+').png')
            m,n=im.shape
            row=np.floor(m/size_image).astype(int)
            col=np.floor(n/size_image).astype(int)
            all_images_array.append(slice_data(im,row,col,size_image))
    else:
        for i in all_images_path:
            im=adjust_image(io.imread(i,as_gray=True))
            m,n=im.shape
            row=np.floor(m/size_image).astype(int)
            col=np.floor(n/size_image).astype(int)
            all_images_array.append(slice_data(im,row,col,size_image))
        

    all_images_array=np.array(all_images_array)
    print(all_images_array.shape)
    all_images_array=np.expand_dims(np.concatenate(all_images_array,axis=0),axis=3)
    print(all_images_array.shape)

    #np.save('E:/pyramidal/jingCode/test',all_images_array)
    return row,col,num_images,all_images_array
def read_image(path,whole_tissue):
    all_images_array=[]
    all_images_path=glob.glob(path)
    num_images=len(all_images_path)
    if whole_tissue:
        for i in range(num_images):
            im=adjust_image(io.imread(path[:-1]+'ori('+str(i+1)+').png',as_gray=True))
    else:
        for i in all_images_path:
            im=adjust_image(io.imread(i,as_gray=True))
    all_images_array.append(im)
    all_images_array=np.array(all_images_array)
    return all_images_array

def reconstruct(row,col,num_image,all_masks_array):
    all_masks_array=np.squeeze(all_masks_array)
    all_image=[]
    for i in range(num_image):
        image=[]
        for j in range(row):
            image.append(np.concatenate(all_masks_array[((i*(row*col))+j*col):((i*(row*col))+(j+1)*col),:],axis=1))
        image=np.array(image)
        image=np.concatenate(image,0)
        print(image.shape)
        all_image.append(image)
    all_image=np.array(all_image)
    print(all_image.shape)
    return all_image
def dilation_erotion(stack_imgs,kernel_size):

    if len(stack_imgs.shape)==4:
        stack_imgs=np.squeeze(stack_imgs)
        
    else:
        dx,dy,dz=stack_imgs.shape
        for i in range(dz):
            print(i)
            img=stack_imgs[:,:,i]
            kernel = np.ones((kernel_size,kernel_size),np.uint8)
            dilation = cv2.dilate(img,kernel,iterations = 1)
            stack_imgs[:,:,i] = cv2.erode(dilation,kernel,iterations = 1)
    return stack_imgs
def precision_recall(ori_mask, pred_mask,save_image):
    ori_mask=np.uint8(ori_mask)
    
    pred_mask=np.uint8(pred_mask)
    
    if np.max(ori_mask)>1:
        ori_mask=ori_mask/255
        ori_mask=np.uint8(ori_mask)
    if np.max(pred_mask)>1:
        pred_mask=pred_mask/255
        pred_mask=np.uint8(pred_mask)
    
    pred_centroid=np.round(cv2.connectedComponentsWithStats(pred_mask)[3]).astype(int)[1:]
    print(pred_centroid.shape)
    ori_centroid=np.round(cv2.connectedComponentsWithStats(ori_mask)[3]).astype(int)[1:]
    print(ori_centroid.shape)
    if save_image:
        
        plt.axis('off')
        plt.imshow(ori_mask,cmap='gray')
        #plt.savefig(str(pred_centroid.shape[0])+'cellsori.png')
        plt.imsave(str(pred_centroid.shape[0])+'cellsori.png',ori_mask,cmap='gray')
        plt.scatter(ori_centroid[:,0],ori_centroid[:,1],s=2,marker='.',c='red')
        plt.axis('off')
        plt.savefig(str(pred_centroid.shape[0])+'cellsoriWithMarker.png')
        plt.show()
        
        plt.axis('off')
        plt.imshow(pred_mask, cmap='gray') 
        #plt.savefig(str(pred_centroid.shape[0])+'cells.png')
        plt.imsave(str(pred_centroid.shape[0])+'cells.png',pred_mask,cmap='gray')
        plt.scatter(pred_centroid[:,0],pred_centroid[:,1],s=2,marker='.',c='red')
        plt.axis('off')
        plt.savefig(str(pred_centroid.shape[0])+'cellsWithMarker.png')
        plt.show()
       
        plt.axis('off')
        plt.imshow(pred_mask, cmap='gray') 
        plt.scatter(ori_centroid[:,0],ori_centroid[:,1],s=2,marker='.',c='red')
        plt.axis('off')
        plt.savefig(str(pred_centroid.shape[0])+'predcellsWithgroundcen.png')
        plt.show()
        
        plt.axis('off')
        plt.imshow(ori_mask,cmap='gray')
        plt.savefig(str(pred_centroid.shape[0])+'cellsori.png')
        plt.scatter(pred_centroid[:,0],pred_centroid[:,1],s=2,marker='.',c='red')
        plt.axis('off')
        plt.savefig(str(pred_centroid.shape[0])+'groundtruthwithpredcen.png')
        plt.show()
   

    #precision
    i=pred_mask[pred_centroid[:,1],pred_centroid[:,0]]
    index=np.where(i==1)[0]
    #predict positive
    tp_fp=np.sum(i[index])
  
    j=ori_mask[pred_centroid[:,1],pred_centroid[:,0]]
    true_positive=np.sum(j)
    precision=true_positive/tp_fp
    
    
    #recall
    #actual positive
    k=pred_mask[ori_centroid[:,1],ori_centroid[:,0]]
    tp_fn=np.sum(k==0)+true_positive
    recall=true_positive/tp_fn
    F1=precision*recall/(precision+recall)
    return precision, recall,F1


def cal_cm(y_true,y_pred):
    y_true=np.array(y_true==0)
    y_pred=np.array(y_pred==0)
    print(y_true.shape,y_pred.shape)
    assert(y_true.shape==y_pred.shape)
    if np.max(y_true)>1:
        y_true=np.uint8(y_true/255)
    if np.max(y_pred)>1:
        y_pred=np.uint8(y_pred/255)
    cm=confusion_matrix(y_true.flatten(),y_pred.flatten())
    return cm
def save_image(image_arrays):
    
    
    image_arrays=np.squeeze(image_arrays)
    num_image=image_arrays.shape[0]
    for i in range(num_image):
        print(i)
        plt.imsave('099pred'+str(i+26)+'.png',image_arrays[i],cmap='gray')


Using TensorFlow backend.


In [0]:
import tensorflow as tf
from keras import backend as K
from keras.losses import binary_crossentropy
from keras import backend as K
from keras import backend 
from keras.models import Model
from keras import models
from keras import layers
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, concatenate, Concatenate, UpSampling2D, Activation
from keras.losses import binary_crossentropy
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from functools import partial
from keras.layers import Lambda
from keras.applications.densenet import DenseNet121
import keras
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
from keras import applications
#%%
#path='E:/pyramidal/jingCode/099_00_800nm_newScanner_Part1_code_data/099_00_800nm_newScanner_Part1/*'
path='drive/test/images/*'
size_image=256
whole_tissue=True
row,col,num_images,all_sub_images=read_images_slice(path,size_image,whole_tissue)
print('We have %d images in total.'%num_images)
print('Each image is split into %d rows and %d columns sub images with size 256*256.'%(row,col))
print(np.max(all_sub_images))
print(num_images)
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def softmax_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) * 0.5 + dice_coef_loss(y_true, y_pred) * 0.5# + dice_coef_loss(y_true[..., 1], y_pred[..., 1]) * 0.2




from keras.models import load_model
model = load_model('05-0.0147.hdf5',custom_objects={'dice_coef_loss':dice_coef_loss,'softmax_dice_loss':softmax_dice_loss})

prediction=model.predict(all_sub_images, batch_size=20, verbose=1)
prediction1=adjust_mask(prediction,0.5)
pred_masks=reconstruct(row,col,num_images,prediction1)
print(pred_masks.shape)
save_image(pred_masks)


# path1='drive/test/masks/*'
# true_label=np.uint8(read_image(path1,False))
# x=cal_cm(true_label,pred_masks)
# print('Confusion Matrix considering shapes')
# print(x)
# save_image=True
# a,b,c=precision_recall (true_label[0],pred_masks[0],save_image) 
# print('Precision, recall and F1 score without considering shapes')
# print(a,b,c)

(25, 272, 256, 256)
(6800, 256, 256, 1)
We have 25 images in total.
Each image is split into 16 rows and 17 columns sub images with size 256*256.
1.0
25


W0707 12:22:59.516247 139898415269760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 12:22:59.681024 139898415269760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 12:22:59.731241 139898415269760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 12:22:59.732601 139898415269760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0707 12:22:59.736957 139898415

6800/6800 [==============================] - 229s 34ms/step
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(25, 4096, 4352)
(25, 4096, 4352)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [0]:
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def softmax_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) * 0.5 + dice_coef_loss(y_true, y_pred) * 0.5# + dice_coef_loss(y_true[..., 1], y_pred[..., 1]) * 0.2




from keras.models import load_model
model = load_model('05-0.0147.hdf5',custom_objects={'dice_coef_loss':dice_coef_loss,'softmax_dice_loss':softmax_dice_loss})

prediction=model.predict(all_sub_images, batch_size=20, verbose=1)
prediction1=adjust_mask(prediction,0.5)
pred_masks=reconstruct(row,col,num_images,prediction1)
print(pred_masks.shape)
save_image(pred_masks)

W0707 11:29:20.731400 140391814342528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 11:29:20.856634 140391814342528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 11:29:20.903691 140391814342528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 11:29:20.905069 140391814342528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0707 11:29:20.909497 140391814

6800/6800 [==============================] - 227s 33ms/step
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(25, 4096, 4352)
(25, 4096, 4352)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [0]:
pred_masks=reconstruct(row,col,num_images,prediction1)
print(pred_masks.shape)
save_image(pred_masks)

(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(4096, 4352)
(25, 4096, 4352)
(25, 4096, 4352)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [0]:
import tensorflow as tf
from keras import backend as K
from keras.losses import binary_crossentropy
from keras import backend as K
from keras import backend 
from keras.models import Model
from keras import models
from keras import layers
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, concatenate, Concatenate, UpSampling2D, Activation
from keras.losses import binary_crossentropy
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from functools import partial
from keras.layers import Lambda
from keras.applications.densenet import DenseNet121
import keras
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
from keras import applications
#%%
#path='E:/pyramidal/jingCode/099_00_800nm_newScanner_Part1_code_data/099_00_800nm_newScanner_Part1/*'
path='drive/test/images/*'
size_image=256
whole_tissue=True
row,col,num_images,all_sub_images=read_images_slice(path,size_image,whole_tissue)
print('We have %d images in total.'%num_images)
print('Each image is split into %d rows and %d columns sub images with size 256*256.'%(row,col))
print(np.max(all_sub_images))
print(num_images)
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def softmax_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) * 0.5 + dice_coef_loss(y_true, y_pred) * 0.5# + dice_coef_loss(y_true[..., 1], y_pred[..., 1]) * 0.2




from keras.models import load_model
model = load_model('30-0.0155.hdf5',custom_objects={'dice_coef_loss':dice_coef_loss,'softmax_dice_loss':softmax_dice_loss})

prediction=model.predict(all_sub_images, batch_size=20, verbose=1)
prediction1=adjust_mask(prediction,0.5)
pred_masks=reconstruct(row,col,num_images,prediction1)
print(pred_masks.shape)
save_image(pred_masks)



(50, 64, 256, 256)
(3200, 256, 256, 1)
We have 50 images in total.
Each image is split into 8 rows and 8 columns sub images with size 256*256.
1.0
50
3200/3200 [==============================] - 3727s 1s/step
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(50, 2048, 2048)
(50, 2048, 2048)


TypeError: ignored